In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

C:\Users\JERRY JOHN\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\JERRY JOHN\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\JERRY JOHN\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is

In [2]:
df = pd.read_csv("autos.csv", header=0, sep=',', encoding='Latin1')
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   dateCrawled          371528 non-null  object
 1   name                 371528 non-null  object
 2   seller               371528 non-null  object
 3   offerType            371528 non-null  object
 4   price                371528 non-null  int64 
 5   abtest               371528 non-null  object
 6   vehicleType          333659 non-null  object
 7   yearOfRegistration   371528 non-null  int64 
 8   gearbox              351319 non-null  object
 9   powerPS              371528 non-null  int64 
 10  model                351044 non-null  object
 11  kilometer            371528 non-null  int64 
 12  monthOfRegistration  371528 non-null  int64 
 13  fuelType             338142 non-null  object
 14  brand                371528 non-null  object
 15  notRepairedDamage    299468 non-nu

In [4]:
df.shape

(371528, 20)

In [49]:
df.isnull().sum()

dateCrawled                0
name                       0
seller                     0
offerType                  0
price                      0
abtest                     0
vehicleType            37869
yearOfRegistration         0
gearbox                20209
powerPS                    0
model                  20484
kilometer                  0
monthOfRegistration        0
fuelType               33386
brand                      0
notRepairedDamage      72060
dateCreated                0
nrOfPictures               0
postalCode                 0
lastSeen                   0
dtype: int64

In [5]:
df=df.drop(['offerType','seller'],axis=1)

In [6]:
#Cars having power less than 50ps and above 900ps seems a little suspicious,
#let's remove them and see what we've got now
df = df[(df.powerPS > 50) & (df.powerPS < 900)]
print(df.shape)
#around 50000 cars ahave been removed which could have inrouduced error to our data

(319709, 18)


In [7]:
#simlarly, filtering our the cars having registeration years not in the mentioned range
#print(df.shape)
df = df[(df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)]
print(df.shape)
# not much of a difference but still, 10000 rows have been reduced. it's better to 
#get rid of faulty data instead of keeping them just to increase the size.

(309171, 18)


In [8]:
#removing irrelevant columns which are either the same for all the cars in teh dataset, or can
#introduce bias, so removing them too.
df.drop(['name','abtest', 'dateCrawled', 'nrOfPictures', 'lastSeen',
         'postalCode', 'dateCreated'], axis='columns', inplace=True)

In [9]:
#dropping the duplicates from the dataframe and stroing it in a new df.
#here all rows having same value in all the mentioned columns will be deleted and by default,
#only first occurance of anysuch row is kept
new_df = df.copy()
new_df = new_df.drop_duplicates(['price','vehicleType','yearOfRegistration'
                         ,'gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType'
                         ,'notRepairedDamage'])

#after removing duplicates
print(new_df.shape)

(285145, 11)


In [10]:
#As the dataset contained some german words for many features, cahnging them to english
new_df.gearbox.replace(('manuell', 'automatik'), ('manual','automatic'), inplace=True)
new_df.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electric'),inplace=True)
new_df.vehicleType.replace(('kleinwagen', 'cabrio','kombi','andere'),
                           ('small car','convertible','combination','others'),inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'),inplace=True)

In [11]:
#### Removing the outliers
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 150000)]

In [12]:
#Filling NaN values for columns whose data might not be there with the information provider,
#which might lead to some variance but our model
#but we will still be able to give some estimate to the user
new_df['notRepairedDamage'].fillna(value='not-declared', inplace=True)
new_df['fuelType'].fillna(value='not-declared', inplace=True)
new_df['gearbox'].fillna(value='not-declared', inplace=True)
new_df['vehicleType'].fillna(value='not-declared', inplace=True)
new_df['model'].fillna(value='not-declared', inplace=True)

In [13]:
new_df.isnull().sum()

price                  0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dtype: int64

In [14]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278578 entries, 1 to 371527
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   price                278578 non-null  int64 
 1   vehicleType          278578 non-null  object
 2   yearOfRegistration   278578 non-null  int64 
 3   gearbox              278578 non-null  object
 4   powerPS              278578 non-null  int64 
 5   model                278578 non-null  object
 6   kilometer            278578 non-null  int64 
 7   monthOfRegistration  278578 non-null  int64 
 8   fuelType             278578 non-null  object
 9   brand                278578 non-null  object
 10  notRepairedDamage    278578 non-null  object
dtypes: int64(5), object(6)
memory usage: 25.5+ MB


In [15]:
# Label Encoding is done on remaining columns

def encoder(data,variable):
    lb=LabelEncoder()
    data[variable]=lb.fit_transform(data[variable])  

In [16]:
encoder(new_df,'vehicleType')
encoder(new_df,'gearbox')
encoder(new_df,'model')
encoder(new_df,'fuelType')
encoder(new_df,'brand')
encoder(new_df,'notRepairedDamage')

In [17]:
new_df.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,3,2011,1,190,162,125000,5,1,1,1
2,9800,8,2004,0,163,118,125000,8,1,14,2
3,1500,7,2001,1,75,117,150000,6,7,38,0
4,3600,7,2008,1,69,102,90000,7,1,31,0
5,650,4,1995,1,102,11,150000,10,7,2,1


In [18]:
x = new_df.drop('price',axis=1)
y = new_df['price']

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 10)

In [20]:
rf=RandomForestRegressor()
rf.fit(x_train,y_train)
ypred=(rf.predict(x_test))
score=r2_score(y_test,ypred)
print('***Random Forest Regressor Model***')
print('Score for Random Forest Regressor Model is {}'.format(score))

C:\Users\JERRY JOHN\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\JERRY JOHN\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


***Random Forest Regressor Model***
Score for Random Forest Regressor Model is 0.8466878407365188


In [29]:
pickle.dump(rf,open("model.pkl","wb"))

In [22]:
x['vehicleType'].value_counts()

4    79547
7    57353
1    56193
0    25469
2    19436
3    15879
8    12389
5    10011
6     2301
Name: vehicleType, dtype: int64

In [23]:
new_row ={'yearOfRegistration': 2015, 'powerPS': 12.0, 'kilometer': 20123.0, 'monthOfRegistration': 2, 'gearbox': 'manual', 'notRepairedDamage': 'No', 'model': '5er', 'brand': 'lada', 'fuelType': 'electric', 'vehicleType': 'car'}
new_row

{'yearOfRegistration': 2015,
 'powerPS': 12.0,
 'kilometer': 20123.0,
 'monthOfRegistration': 2,
 'gearbox': 'manual',
 'notRepairedDamage': 'No',
 'model': '5er',
 'brand': 'lada',
 'fuelType': 'electric',
 'vehicleType': 'car'}

In [24]:
new_df = pd.DataFrame(columns =['vehicleType', 'yearOfRegistration', 'gearbox',
                                'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType',
                                'brand', 'notRepairedDamage'] )

In [25]:
new_df = new_df.append(new_row,ignore_index = True)

In [26]:
new_df

,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,car,2015,manual,12.0,5er,20123.0,2,electric,lada,No


In [27]:
rf.predict(new_df)

ValueError: could not convert string to float: 'car'

In [28]:
new_df = pd.DataFrame(columns =['vehicleType', 'yearOfRegistration', 'gearbox',
                                'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType',
                                'brand', 'notRepairedDamage'] )

new_df = new_df.append(new_row,ignore_index = True)

def encoder(data,variable):
    lb=LabelEncoder()
    data[variable]=lb.fit_transform(data[variable])  
    
encoder(new_df,'vehicleType')
encoder(new_df,'gearbox')
encoder(new_df,'model')
encoder(new_df,'fuelType')
encoder(new_df,'brand')
encoder(new_df,'notRepairedDamage')    

rf.predict(new_df)

C:\Users\JERRY JOHN\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


array([6306.2])